<a href="https://colab.research.google.com/github/MedinBarroso/Modelo-de-ML-para-la-clasificaci-n-de-RNFs/blob/master/Algoritmo_de_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Importamos las librerias 
import spacy
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

#Importamos el dataset con pandas

url = 'https://raw.githubusercontent.com/MedinBarroso/Modelo-de-ML-para-la-clasificaci-n-de-RNFs/588ddbd3ec146a13bdac4c7841da2eea9d4414f9/pfix.csv'


dframe = pd.read_csv(url)
dframe.head()

,Project.ID,Requirement.description,NFR
0,1,'The system shall refresh the display every 60...,PE
1,1,'The application shall match the color of the ...,US
2,1,' If projected the data must be readable. On...,US
3,1,' The product shall be available during normal...,RE
4,1,' If projected the data must be understandabl...,US


In [ ]:
dframe.shape

(526, 3)

In [ ]:
dframe.NFR.value_counts()

SE    151
US    118
F     115
CO     48
PE     39
RE     27
MN     15
PO     13
Name: NFR, dtype: int64

In [ ]:
#Importamos un modelo de ingles del spacy

import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
#nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_trf')

# armamos una lista de stopwords para filtrarlas
stopwords = list(STOP_WORDS)

print(stopwords[:100])


NameError: ignored

In [ ]:
import string
import spacy
punctuations = string.punctuation
# crear un Spacy Parser
from spacy.lang.en import English
parser = English()


In [ ]:
def my_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens




In [ ]:
# Librerias de ML
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier

In [ ]:
#Custom transformer usando spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Funcion basica para limpiar txto 
def clean_text(text):     
    return text.strip().lower()

In [ ]:
# Vectorizacion (solo se usa un clasificador a la vez, por eso los otros estan con comentarios)
vectorizer = CountVectorizer(tokenizer = my_tokenizer, ngram_range=(1,2)) 
classifier = LinearSVC()
#classifier =  MultinomialNB()
#classifier = LogisticRegression()
#classifier = RandomForestClassifier (n_estimators=100, max_depth=5, random_state=0)
print(vectorizer)


In [ ]:
# Dividimos el dataset
from sklearn.model_selection import train_test_split

In [ ]:
# Caracterisiticas y etiquetas
X = dframe['Requirement.description']
ylabels = dframe['NFR']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.40, random_state=53)

#print (X_train)
#print (ylabels)


In [ ]:
from sklearn import metrics
# Creamos un pipeline para limpiar, vectorizar y clasificar usando"Count Vectorizor"
pipe_countvect = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

# Ajustamos los datos
pipe_countvect.fit(X_train,y_train)
# Predecimos con un dataset de prueba
sample_prediction = pipe_countvect.predict(X_test)

# Resultado de las predicciones
#Imprimimos los resultados de las predicciones
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)
    

#print("Accuracy: ",pipe_countvect.score(X_test,y_test))
#print("Accuracy: ",pipe_countvect.score(X_test,sample_prediction))
# Accuracy
#print("Accuracy: ",pipe_countvect.score(X_train,y_train))

In [ ]:
# Accuracy
print('Accuracy',metrics.accuracy_score(y_test,sample_prediction))
#Precision
print('Precision',metrics.precision_score(y_test,sample_prediction, average='macro'))
#Precision
print('Recall',metrics.recall_score(y_test,sample_prediction, average='macro'))

pre = metrics.precision_score(y_test,sample_prediction, average='macro')
re = metrics.recall_score(y_test,sample_prediction, average='macro')
f1 = 2*(pre*re)/(pre+re)

#F1 = 2 * (precision * recall) / (precision + recall)
print('F1 Score',f1)

from sklearn.metrics import classification_report
print(classification_report(y_test, sample_prediction))


In [ ]:
# Oración para pruebas
pipe_countvect.predict(["the transaction should be completed in less than 10 seconds"])

In [ ]:
#Graficos

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, sample_prediction, normalize=False)
plt.show()

In [ ]:
plt.scatter(y_test, sample_prediction)
plt.show()